In [1]:
# ====================================================================
# FULL PROJECT ANALYSIS: DATA PREPARATION, EDA, AND GARCH STRATEGY BACKTEST
#
# This script performs the following tasks:
# 1. Loads raw tick data files and combines them (Data Preparation).
# 2. Filters the combined data to the project period and calculates mid-price.
# 3. Executes Exploratory Data Analysis (EDA) on the tick data.
# 4. Implements the GARCH-Adaptive Volatility Breakout Strategy (Lecture 3).
# 5. Calculates Net P&L, Sharpe Ratio, and Max Drawdown (Lecture 2).
# ====================================================================

# --- DEPENDENCIES ---
!pip install arch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from arch import arch_model
from datetime import datetime, timezone
import os

# --- GLOBAL CONFIGURATION ---
# Data Input Paths (from Google Colab environment)
FILE_PATH_SEP = '/content/EURPLN-2025-09.csv'
FILE_PATH_OCT = '/content/EURPLN-2025-10.csv'

# Project Timeline
START_PERIOD = datetime(2025, 9, 1, 0, 0, 0, tzinfo=timezone.utc) # Start of full dataset
END_PERIOD = datetime(2025, 10, 6, 0, 0, 0, tzinfo=timezone.utc)   # End of full dataset

# Strategy Split Point (Start of Out-of-Sample/Testing Period)
SPLIT_DATE = datetime(2025, 9, 22, 0, 0, 0, tzinfo=timezone.utc)

# Output filename for the combined, cleaned data (for user validation)
CLEAN_OUTPUT_FILE = '/content/EURPLN_01Sep_06Oct_clean_tick_data.csv'

# Strategy Parameters
K_FACTOR = 3.0       # Volatility scaling factor for breakout threshold
COST_BPS = 0.0001    # Estimated transaction cost (1 basis point or 1 pip spread)
# ====================================================================


# ====================================================================
# SECTION 1: DATA PREPARATION AND EXPLORATORY DATA ANALYSIS (EDA)
# ====================================================================

def load_and_combine_data():
    """Loads, combines, cleans, and exports the final tick data."""
    print("--- 1.1 Starting Data Loading and Combination ---")

    def load_file(filepath):
        # Reads the CSV, assuming a standard tick format: [Time, Bid, Ask, ...]
        try:
            # === ROBUST LOADING ASSUMPTION (LIKELY FIX) ===
            # Assume no header row (header=None) and that the timestamp is the first column (index_col=0).
            # The columns are named generically and we select the Bid (V1) and Ask (V2) columns.
            print(f"Loading {os.path.basename(filepath)} assuming no header (Time, Bid, Ask, ...)")
            df = pd.read_csv(
                filepath,
                index_col=0,
                parse_dates=True,
                header=None, # Assume NO HEADER
                names=['Time', 'Bid', 'Ask', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8'] # Assign generic names
            )
            df.index.name = 'Time'

            # Select only the critical Bid and Ask columns
            df = df[['Bid', 'Ask']].copy()

            print(f"Loaded {len(df)} ticks from {os.path.basename(filepath)}")

            # Print column types and first few rows for debugging if issues persist
            # print(f"Sample data head:\n{df.head()}")
            # print(f"Index type: {df.index.dtype}")

            return df

        except Exception as e:
            # Print detailed error to help diagnose issues if the format is truly non-standard
            print(f"CRITICAL ERROR loading {filepath}: {e}")
            print("--- DATA LOADING FAILED ---")
            print("If the error persists, please check your CSV structure:")
            print("1. Is the timestamp in the FIRST column?")
            print("2. Are the Bid and Ask prices in the SECOND and THIRD columns, respectively?")
            print("3. Is the date format consistent (e.g., YYYY-MM-DD HH:MM:SS.mmm)?")
            return pd.DataFrame()

    # Load and combine both files
    df_sep = load_file(FILE_PATH_SEP)
    df_oct = load_file(FILE_PATH_OCT)

    if df_sep.empty or df_oct.empty:
        # Re-raise error if data frames are still empty after loading
        raise ValueError("One or both data files failed to load. Check file paths and column structure.")

    combined_df = pd.concat([df_sep, df_oct]).sort_index()
    print(f"Total combined ticks before filtering: {len(combined_df)}")

    # Calculate Mid Price
    combined_df['Price'] = (combined_df['Bid'] + combined_df['Ask']) / 2

    # Filter to exact period and clean
    if combined_df.index.tz is None:
        combined_df = combined_df.tz_localize(timezone.utc)

    filtered_df = combined_df.loc[
        (combined_df.index >= START_PERIOD) &
        (combined_df.index < END_PERIOD)
    ].copy()

    filtered_df = filtered_df[~filtered_df.index.duplicated(keep='first')]
    filtered_df.dropna(subset=['Price'], inplace=True)

    print(f"Final Ticks for Project ({START_PERIOD.strftime('%Y-%m-%d %H:%M:%S')} to {END_PERIOD.strftime('%Y-%m-%d %H:%M:%S')}): {len(filtered_df)}")

    # Export for Validation
    filtered_df[['Price', 'Bid', 'Ask']].to_csv(CLEAN_OUTPUT_FILE)
    print(f"\nSUCCESS: Cleaned data exported to: {CLEAN_OUTPUT_FILE} (for user review)")

    return filtered_df

def run_eda(data):
    """Performs basic Exploratory Data Analysis on the prepared tick data."""
    print("\n--- 1.2 Exploratory Data Analysis (EDA) ---")

    # Summary Statistics
    print("\nSummary Statistics of Tick Prices:")
    print(data['Price'].describe())

    # Calculate Spread (Ask - Bid) in basis points (BPS)
    data['Spread_BPS'] = (data['Ask'] - data['Bid']) * 10000

    # Spread Analysis
    print("\nTick Spread Statistics (in BPS):")
    print(data['Spread_BPS'].describe())

    # Plotting Price Evolution (Downsampled for performance)
    plt.figure(figsize=(12, 6))
    data['Price'].resample('1H').last().plot(
        title='EUR/PLN Price Evolution (Hourly Resampled)',
        ylabel='Price',
        color='blue'
    )
    plt.grid(True, alpha=0.3)
    plt.show()
    #

    # Plotting Spread Distribution
    plt.figure(figsize=(8, 5))
    data['Spread_BPS'].plot(
        kind='hist',
        bins=50,
        title='Distribution of Tick Spreads (in BPS)',
        xlabel='Spread (BPS)',
        color='red',
        alpha=0.7
    )
    plt.grid(True, alpha=0.3)
    plt.show()
    #


# ====================================================================
# SECTION 2: GARCH STRATEGY IMPLEMENTATION AND BACKTEST
# ====================================================================

def run_garch_strategy(data_full):
    """
    Implements the GARCH-Adaptive Volatility Breakout strategy (Lecture 3).
    """
    print("\n--- 2.1 Starting GARCH Strategy Backtest ---")

    # 1. Calculate Log Returns
    data_full['Log_Return'] = np.log(data_full['Price'] / data_full['Price'].shift(1))
    data_full.dropna(subset=['Log_Return'], inplace=True)

    # 2. Split Data Chronologically
    train_data = data_full.loc[data_full.index < SPLIT_DATE].copy()
    test_data = data_full.loc[(data_full.index >= SPLIT_DATE) & (data_full.index < END_PERIOD)].copy()

    print(f"\nTraining Ticks (In-Sample): {len(train_data)}")
    print(f"Testing Ticks (Out-of-Sample): {len(test_data)}")

    if test_data.empty:
        print("ERROR: Testing data is empty. Check split dates and input file range.")
        return

    # 3. GARCH MODEL FITTING (Core Statistical Tool - Lecture 3)
    # Fit GARCH(1,1) on 1-minute bars for computational stability
    print("Fitting GARCH(1,1) Model on 1-Minute resampled training data...")
    train_resampled = train_data['Price'].resample('1T').last().dropna()
    train_resampled_returns = np.log(train_resampled / train_resampled.shift(1)).dropna()

    # Scale returns by 1000 for better optimization
    model = arch_model(train_resampled_returns * 1000, vol='Garch', p=1, q=1, mean='Zero', dist='Normal')
    res = model.fit(disp='off')

    # 4. VOLATILITY CALCULATION (Adaptive Band)
    # Use Rolling Volatility on the Ticks as the adaptive band based on observed high-frequency noise.
    # WINDOW_SIZE_TICKS = 15000 (approx. 1 hour of trading data)
    WINDOW_SIZE_TICKS = 15000

    # Apply rolling volatility to the entire dataset (including training data for lookback)
    data_full['Rolling_Vol'] = data_full['Log_Return'].rolling(window=WINDOW_SIZE_TICKS).std()

    # 5. SIGNAL GENERATION (Tick-by-Tick Breakout)
    test_data['Rolling_Vol'] = data_full['Rolling_Vol'].loc[test_data.index]
    test_data.dropna(subset=['Rolling_Vol'], inplace=True)

    # Adaptive Breakout Bands
    upper_band = test_data['Rolling_Vol'] * K_FACTOR
    lower_band = -test_data['Rolling_Vol'] * K_FACTOR

    test_data['Signal'] = 0
    # Entry: Go Long (1) if the return breaks above the adaptive upper band
    test_data.loc[test_data['Log_Return'] > upper_band, 'Signal'] = 1
    # Entry: Go Short (-1) if the return breaks below the adaptive lower band
    test_data.loc[test_data['Log_Return'] < lower_band, 'Signal'] = -1

    # Position Management: Hold until a counter-signal is received
    test_data['Position'] = test_data['Signal'].replace(0, np.nan).ffill().fillna(0)

    # 6. P&L CALCULATION (Lecture 2)
    test_data['Strategy_Ret_Gross'] = test_data['Position'].shift(1) * test_data['Log_Return']

    # Net P&L: Gross PnL minus transaction costs
    test_data['Turnover'] = test_data['Position'].diff().abs()
    test_data['Transaction_Cost'] = test_data['Turnover'] * COST_BPS
    test_data['Strategy_Ret_Net'] = test_data['Strategy_Ret_Gross'] - test_data['Transaction_Cost']

    # Cumulative P&L
    test_data['Cum_Gross_PnL'] = test_data['Strategy_Ret_Gross'].cumsum()
    test_data['Cum_Net_PnL'] = test_data['Strategy_Ret_Net'].cumsum()

    test_data.dropna(subset=['Cum_Net_PnL'], inplace=True)

    # 7. PERFORMANCE METRICS (Lecture 2)
    total_gross_return = test_data['Cum_Gross_PnL'].iloc[-1]
    total_net_return = test_data['Cum_Net_PnL'].iloc[-1]

    # Annualization Factor for Tick Data (252*24*3600 periods per year)
    annualization_factor = np.sqrt(252 * 24 * 3600)

    if test_data['Strategy_Ret_Net'].std() == 0:
        sharpe_ratio = 0
    else:
        # Sharpe Ratio
        sharpe_ratio = (test_data['Strategy_Ret_Net'].mean() / test_data['Strategy_Ret_Net'].std()) * annualization_factor

    # Max Drawdown
    cum_ret = (1 + test_data['Strategy_Ret_Net']).cumprod()
    peak = cum_ret.cummax()
    drawdown = (cum_ret - peak) / peak
    max_drawdown = drawdown.min() if not drawdown.empty else 0

    # 8. Output Metrics
    print("\n" + "="*40)
    print("   STRATEGY PERFORMANCE (OUT-OF-SAMPLE)")
    print("="*40)
    print(f"Total Gross P&L:     {total_gross_return:.4f}")
    print(f"Total Net P&L (Profit):{total_net_return:.4f}")
    print(f"Annualized Sharpe Ratio:{sharpe_ratio:.4f}")
    print(f"Max Drawdown:        {max_drawdown:.2%}")
    print("="*40)

    # 9. Plotting
    plt.figure(figsize=(12, 6))
    plt.plot(test_data.index, test_data['Cum_Net_PnL'], label='Net P&L (Tick-by-Tick)', color='#10B981')
    plt.title('GARCH-Adaptive Volatility Breakout: Out-of-Sample Net P&L')
    plt.xlabel('Date')
    plt.ylabel('Cumulative Log Return')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.show()
    #


# ====================================================================
# MAIN EXECUTION FLOW
# ====================================================================
if __name__ == '__main__':
    # 1. Data Preparation and Loading
    try:
        clean_tick_data = load_and_combine_data()
    except Exception as e:
        print(f"\nFATAL ERROR: Could not prepare data. {e}")
        clean_tick_data = pd.DataFrame()

    if clean_tick_data.empty:
        print("Execution halted due to data errors.")
    else:
        # 2. Exploratory Data Analysis
        run_eda(clean_tick_data)

        # 3. Strategy Backtest
        run_garch_strategy(clean_tick_data)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.3/981.3 kB 22.5 MB/s eta 0:00:00
--- 1.1 Starting Data Loading and Combination ---
Loading EURPLN-2025-09.csv assuming no header (Time, Bid, Ask, ...)
Loaded 2811673 ticks from EURPLN-2025-09.csv
Loading EURPLN-2025-10.csv assuming no header (Time, Bid, Ask, ...)
Loaded 2583078 ticks from EURPLN-2025-10.csv
Total combined ticks before filtering: 5394751

FATAL ERROR: Could not prepare data. can only concatenate str (not "float") to str
Execution halted due to data errors.
